<div class="alert alert-block alert-success">

# <center> LinkedIn Automate </center>
## <center>  Connection Requests </center>
    
### <center>  for  </center>
## <center>  2nd+ and 3rd+ </center>
<br>
    

### <center>  Connections ranging from 2,3 and 4</center>
    

</div>

In [ ]:
!pip install selenium

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [ ]:
# LinkedIn login credentials
username = 'xxxxxx@gmail.com'
password = 'Xxxxxx123@'

In [ ]:
# Read LinkedIn profile URLs from Excel file
excel_file_path = 'path_to_your_excel_file.xlsx'  # Replace with your actual file path
df = pd.read_excel(excel_file_path)
profiles = df['person_linkedin_url'].tolist()

In [ ]:
# Personal note to include in the invitation
personal_note = "Hi, I came across your profile and would love to connect with you."

In [ ]:
# Set up the WebDriver with headless mode
driver_path = r'C:\Users\chand\OneDrive\Documents\LinkedIn_Automation\edgedriver_win32\msedgedriver.exe'
service = Service(driver_path)
options = Options()
options.headless = True  # Enable headless mode

driver = webdriver.Edge(service=service, options=options)
wait = WebDriverWait(driver, 10)

In [ ]:
# Function to login to LinkedIn
def login(driver, username, password):
    driver.get('https://www.linkedin.com/login')
    time.sleep(2)
    driver.find_element(By.ID, 'username').send_keys(username)
    driver.find_element(By.ID, 'password').send_keys(password)
    driver.find_element(By.XPATH, '//button[@type="submit"]').click()
    time.sleep(5)  # Adjust this sleep time as needed

In [ ]:
# Function to send an invitation
def send_invitation(driver, profile_url, note):
    driver.get(profile_url)
    time.sleep(5)  # Adjust this sleep time as needed
    try:
        # Ensure the page is fully loaded
        driver.execute_script("return document.readyState") == "complete"
        time.sleep(2)

        # Try different locators for the Connect button
        connect_button = None
        connect_locators = [
            '//button[contains(@aria-label, "Connect")]',
            '//button[contains(text(), "Connect")]',
            '//button[contains(@aria-label, "Connect with")]',
            '//button[contains(@aria-label, "Invite")]',
            '//button[contains(text(), "Invite")]',
        ]

        for locator in connect_locators:
            try:
                connect_button = wait.until(EC.element_to_be_clickable((By.XPATH, locator)))
                if connect_button:
                    print(f"Connect button found with locator: {locator}, clicking...")
                    driver.execute_script("arguments[0].click();", connect_button)
                    time.sleep(2)
                    break
            except:
                print(f"Connect button with locator: {locator} not found, trying next...")

        if not connect_button:
            print(f"Error: Connect button not found for {profile_url}")
            return

        # Add a note
        try:
            add_note_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Add a note"]')))
            print("Add note button found, clicking...")
            driver.execute_script("arguments[0].click();", add_note_button)
            time.sleep(2)

            text_area = driver.find_element(By.XPATH, '//textarea[@name="message"]')
            print("Message text area found, entering note...")
            text_area.send_keys(note)

            send_button = driver.find_element(By.XPATH, '//button[@aria-label="Send now"]')
            print("Send button found, clicking...")
            driver.execute_script("arguments[0].click();", send_button)
            time.sleep(2)
        except Exception as e:
            print(f"Add note button not found or error occurred: {e}")
            print("Attempting to send invitation without note...")
            try:
                send_button = driver.find_element(By.XPATH, '//button[@aria-label="Send now"]')
                driver.execute_script("arguments[0].click();", send_button)
                time.sleep(2)
            except Exception as inner_e:
                print(f"Error sending invitation without note: {inner_e}")
    except Exception as e:
        print(f"Error sending invitation to {profile_url}: {e}")

In [ ]:
# Main script
login(driver, username, password)
for profile in profiles:
    send_invitation(driver, profile, personal_note)

# Close the browser
driver.quit()